In [7]:
import math

tsw_min = 37.5 * 1e-9
tsw_max = 107.5 * 1e-9
tsw_avr = 72.5 * 1e-9
trti_min = 297.5 * 1e-9
trti_max = 367.5 * 1e-9
trti_avr = 332.5 * 1e-9
trefi = 3900 * 1e-9
sin_max = 1.28 * 1e10 / 64

sout = 5 * 1e9 * 2 / 64
# sout = 4.95 * 1e9 * 2 / 64
# sout = 4.9 * 1e9 * 2 / 64
# sout = 4.85 * 1e9 * 2 / 64
# sout = 4.8 * 1e9 * 2 / 64
# sout = 4.75 * 1e9 * 2 / 64
# sout = 4.7 * 1e9 * 2 / 64

kn = (trefi - trti_avr - sout/sin_max * trefi) / tsw_avr
kn

7.1810344827586166

In [8]:
nseq_min = math.ceil((trefi - trti_avr - kn * tsw_avr) / kn * sin_max)
nseq_min 

85

In [9]:
import math

nFifo = math.ceil(2*tsw_max*sout + trti_max * sout + tsw_max * sin_max)
nFifo

113

In [10]:
math.ceil(2*tsw_avr*sout + trti_avr * sout + tsw_avr * sin_max)

90

In [11]:
fifo_in_interval = 1 / sin_max
fifo_in_interval

5e-09

In [12]:
fifo_out_interval = 1 / sout
fifo_out_interval

6.4e-09

In [21]:
from sdram1 import Sdram1
from fifo import Fifo
from clock1 import Clock1
import random

tstart = random.uniform(0, trefi)
ddr_sdram = Sdram1([nseq_min])
fifo = Fifo(nFifo)
clock = Clock1(fifo_in_interval, fifo_out_interval,
               trefi, tsw_avr, trti_avr, tstart)

with open("./TestFifoModule", "w") as f:
    print("tstart: %12.6fns" %
            (tstart * 1e9), file=f)

    while(1):
        clock.run()
        fin = clock.fifo_in_finish()
        ffull = fifo.is_full()
        fout = clock.fifo_out_finish()
        dsw = clock.is_sw()

        if fin and (not ffull):
            ddr_sdram.send_out()
            fifo.receive_in()

            if ddr_sdram.get_in_sw():
                clock.set_sw()

        if fout:
            fifo.send_out()

        cur_fifo_n = fifo.get_n_64byte_fifo_current()
        t_in_refi = clock.get_current_t_in_refi()
        cur_wfm_64byte = ddr_sdram.get_current_wfm_64byte()
        in_rti = clock.is_rti()

        # 打印信息
        print("t: %12.6fns" %
            (t_in_refi * 1e9), end="\t", file=f)
        print("fin: {:3}\t fout: {:3}\t dsw: {:3}".format(
            fin, fout, dsw), end="\t", file=f)
        print("rti: {:3}".format(in_rti), end="\t", file=f)
        print(
            "fifo_n: {:5}\t n_cycle: {:4}\t wfm_64byte: {:5}".
            format(
                cur_fifo_n, clock.get_current_cycle_num(),
                cur_wfm_64byte),
            file=f
        )


FifoEmpty: 